[View in Colaboratory](https://colab.research.google.com/github/fevzidas/deeplearning/blob/master/mnist_one_layer01.ipynb)

In [7]:
# Udemy Bölüm 4, Ders 15

# Tensorflow Google tarafından otomatik olarak import edildiği için tekrar yazmıyoruz. 
# One host her bir sınıfın 0 ve 1 lerden oluşan bir vektör ile gösterilmesini sağlamaktadır.
# Örneğin ağaç, ev, araba gibi 3 sınıf için
# ağaç = [1,0,0], ev = [0,1,0] ve araba = [0,0,1] 
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/MNIST', one_hot=True)


# Placeholder (Yer tutucu) tanımlama
# Gelecek olan veriler float32 türündedir. 
# None kaç adet resim geleceğini belirtir.
# 784 ise işlenecek her bir resmin vektöre dönüştürülmüş boyutu (28x28) 
# Placeholder şu anda boş

x = tf.placeholder(tf.float32, [None, 784])


# İşlenecek tüm resimlerin labelleri vardır. Bu labeller için yer tutucu tanımluyoruz.
# MNIST resimleri 0-9 aralığındaki toplam 10 adet rakamdan oluşmaktadır. Yani label sayısı 10 dur.
# Dolayısıyla gerçek etiket değerlerini tutan y_true yer tutucusunun boyutu 10 olarak belirlenmiştir. 
y_true = tf.placeholder(tf.float32, [None, 10])

# Genel formülümüz y = w.x + b şeklindeydi. Yukarıda x tanımlandı. 
# Şimdi sıra diğer değişkenlerde
# Weight tanımı
# Variable eğitilecek değerler olduğunu gösterir.
# x ve w çarpımı yapabilmek için satır sütün sayısı eşit olmalıdır.
# Mevcut durumda x = [None, 784] tür. 
# O halde w = [784, 10] olmalıdır. 10 nöron (sınıf) sayısını göstermektedir. 
# Örneğin x = 500 resim verildiğinde x=[500,784], w=[784,10] ve x*w=[500,10] olacaktır.
# Yani 500 resmin her biri için ayrı ayrı 10 tahmin yapılacaktır.
# Bu yapıda tek layerli bir network oluşturulmuştur.
# Bu layersa 10 nöron vardır. Bu nöronlar her bir resim için sınıfı tahmin etmeye çalışacaktır. 
w = tf.Variable(tf.zeros([784, 10]))

# Geriye bayes yani b kaldı.
# 10 nöron olduğu için 10 değeri verildi.
b = tf.Variable(tf.zeros([10]))

# w.x + b ifadesine genel olarak logist denir.
# w ve b ise optimize edilecek olan değerlerdir. Bu değerlere başlangıçta 0 değerleri verilmiştir.
logits = tf.matmul(x, w) + b

# logist değerini aktivasyon fonksiyonundn geçirme işlemi
# softmax ile tüm değerler 0-1 aralığına sıkıştırılır.
# Tüm değerlerin toplamı ise 1 dir. (olasılık gibi)
# Tensorflowda aktivasyon fonksiyonları nn kütüphanesinde yer alır.
# logits = [1.2, 0.6, -0.4, -0.2, 4.3, 0.1, 0.3, -0.1, 0.5, 0.7]
# softmax(logits) = [0.04, 0.02, 0.01, 0.01, 0.84, 0.01, 0.02, 0.01, 0.02, 0.02]
y = tf.nn.softmax(logits)


# Şimdi modelin doğruluğunu kontrol edelim
# Bunun için loss fonksiyonu kullanılır.
# Bu çalışmada loss olarak cross entropy fonksiyonu kullanılacaktır.
# Cross entropy iki parametre istemektedir. Birincisi tahmin edilen değer olan logits = tf.matmul(x, w) + b
# İkincisi ise nesnelere ait gerçek sınıf değerlerini içeren labels değeri
# Logist değeri olarak y yerine aktivasyon fonksiyonundan geçmemiş olan logits değeri verilecektir.
# Çünkü cross entropy kendi içinde aktivasyon fonksiyonuna zaten sahiptir. 
xent = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)

# xent ortalamaları alınacak olan değerleri içeren vektördür. 
# Örneğin batch size 128 ise bu durumda xent 128 değer tutan bir vektör demektir. 
loss = tf.reduce_mean(xent)

# Modelin doğruluğunu hesaplama
# y değeri tahmin edilen değeri y_true ise gerçek değer tutmaktadır
# 1 değeri dimension (boyut) bilgisini verir
# argmax fonksiyonu  en büyük değeri olan endeksi döndürür.
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_true, 1))

#correct_prediction True ve False değerlerini içeren bir vekördür.
# Bu vektöründeki değerlerin ortalamasını almak için cast ile tür dönüşümü yapılmalıdır.
# Kullanımı cast(dönüştürülecek_değer, dönüştürülecek_tip)
# reduce mean ise ortalama almaktadır. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Optimizasyon işlemi
# 0.5 learning rate 
optimize = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Bu kodları çalıştırmak için Oturum açılmalıdır
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Her yığında olması gereken resim sayısı
batch_size = 128


# iterations kaç defa döngü dönüp eğitim yapılacağını belirtir.
def training_step(iterations):
  for i in range(iterations):
    #MNIST data setinden belirlenen sayıda (batch_size) rastgelene resim alınmasını sağlar.
    # x_batch resimleri, y_batch resimlere ait etiket (label) değerlerini tutar
    x_batch, y_batch = mnist.train.next_batch(batch_size)
    feed_dict_train = {x: x_batch, y_true: y_batch}
    sess.run(optimize, feed_dict=feed_dict_train)

def test_accuracy():
  feed_dict_test = {x:mnist.test.images, y_true:mnist.test.labels }
  acc = sess.run(accuracy, feed_dict=feed_dict_test)
  print('Testing accuracy : ', acc)


training_step(600)
test_accuracy()

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Testing accuracy :  0.9174
